## Waste Management Optimization Model
##### Waste is generated in two centers: New York and New Jersey. It can then be transpoted to four possible transfer station depots: Bronx, Brooklyn, Queens, and Staten Island. 
##### It can also be sent directly to one of the six landfills: C1- C6. 
##### The constraints to be satisfied in this problem are related to the fact that (1) there are limits to the maximum volume of waste that can pass through each depot; (2) each generation center has a limit to the maximum waste it can generate; (3) each landfill has a known demand
##### There are also transportation costs associated with each possible route. 
##### Below are the specifications for each of the constraints and the costs. 

| Center | Waste (tons) |
| --- | --- |
| NewYork | 300,000 |
| NewJersey |  400,000 |

The waste can be shipped from a center to a set of four depots. Each depot has a maximum throughput.

| Depot | Throughput (tons) |
| --- | --- |
| Bronx | 140,000 |
| Brooklyn | 100,000 |
| Queens | 200,000 |
| StatenIsland | 80,000 |

Our network has six landfills, each with a given maxiumum demand.

| Landfill | Demand (tons) |
| --- | --- |
| C1 | 100,000 |
| C2 | 20,000 |
| C3 | 80,000 |
| C4 | 70,000 |
| C5 | 120,000 |
| C6 | 40,000 |

Transporation costs are given in the following table (in dollars per ton).  Columns are source cities and rows are destination cities.  

| To | NewYork | NewJersey | Bronx | Brooklyn | Queens | StatenIsland |
| --- | --- | --- | --- | --- | --- | --- |
| Depots |
| Bronx  | 0.7 |   - |
| Brooklyn | 0.7 | 0.5 |
| Queens     | 1.2 | 0.7 |
| StatenIsland     | 0.4 | 0.4 |
| Landfills |
| C1 | 1.2 | 2.2 |   - | 1.2 |   - |   - |
| C2 |   - |   - | 1.7 | 0.7 | 1.7 |   - |
| C3 | 1.7 |   - | 0.7 | 0.75 | 2.2 | 0.4 |
| C4 | 2.2 |   - | 1.7 | 1.2|   - | 1.7 |
| C5 |   - |   - |   - | 0.7 | 0.7 | 0.7 |
| C6 | 1.2 |   - | 1.2 |   - | 1.7 | 1.7 |

Question: How to satisfy the demands of the end Landfills while minimizing shipping costs.

---

### Model Formulation

#### Sets and Indices

$f \in \text{Centers}=\{\text{NewYork}, \text{NewJersey}\}$

$d \in \text{Depots}=\{\text{Bronx}, \text{Brooklyn}, \text{Queens}, \text{StatenIsland}\}$

$c \in \text{Landfills}=\{\text{C1}, \text{C2}, \text{C3}, \text{C4}, \text{C5}, \text{C6}\}$

$\text{Locations} = \text{Centers} \cup \text{Depots} \cup \text{Landfills}$

#### Parameters

$\text{cost}_{s,t} \in \mathbb{R}^+$: Cost of shipping one ton from source $s$ to destination $t$.

$\text{supply}_f \in \mathbb{R}^+$: Maximum possible supply from center $f$ (in tons).

$\text{through}_d \in \mathbb{R}^+$: Maximum possible flow through depot $d$ (in tons).

$\text{demand}_c \in \mathbb{R}^+$: Demand for waste at landfill $c$ (in tons).

#### Decision Variables

$\text{flow}_{s,t} \in \mathbb{N}^+$: Quantity of waste (in tons) that is shipped from source $s$ to destionation $t$.


#### Objective Function

- **Cost**: Minimize total shipping costs.

\begin{equation}
\text{Minimize} \quad Z = \sum_{(s,t) \in \text{locations} \times \text{locations}}{\text{cost}_{s,t}*\text{flow}_{s,t}}
\end{equation}

#### Constraints

- **Center output**: Flow of goods from a factory must respect maximum capacity.

\begin{equation}
\sum_{t \in \text{locations}}{\text{flow}_{f,t}} \leq \text{supply}_{f} \quad \forall f \in \text{Centers}
\end{equation}

- **Landfill demand**: Flow of goods must meet customer demand.

\begin{equation}
\sum_{s \in \text{locations}}{\text{flow}_{s,c}} = \text{demand}_{c} \quad \forall c \in \text{Landfills}
\end{equation}

- **Depot flow**: Flow into a depot equals flow out of the depot.

\begin{equation}
\sum_{s \in \text{locations}}{\text{flow}_{s,d}} = 
\sum_{t \in \text{locations}}{\text{flow}_{d,t}}
\quad \forall d \in \text{Depots}
\end{equation}

- **Depot capacity**: Flow into a depot must respect depot capacity.

\begin{equation}
\sum_{s \in \text{locations}}{\text{flow}_{s,d}} \leq \text{through}_{d}
\quad \forall d \in \text{Depots}
\end{equation}

---

### Python Implementation

In [4]:
#import needed packages
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

#### Define Input Data

In [10]:
#create dictionaries to define data for supply limits, depots limits,landfill demands, and shipping costs
supply = dict({"New York": 300000,
              "New Jersey": 400000})
through = dict({ "Bronx": 140000,
             "Brooklyn": 100000,
             "Queens": 200000,
             "StatenIsland": 80000})
demand = dict({"C1": 100000,
              "C2": 20000,
              "C3": 80000,
              "C4": 70000,
              "C5": 120000,
              "C6": 40000})
arcs, cost = gp.multidict({
    ('NewYork', 'Bronx'): 0.7,
    ('NewYork', 'Brooklyn'): 0.7,
    ('NewYork', 'Queens'): 1.2,
    ('NewYork', 'StatenIsland'): 0.4,
    ('NewYork', 'C1'): 1.2,
    ('NewYork', 'C3'): 1.7,
    ('NewYork', 'C4'): 2.2,
    ('NewYork', 'C6'): 1.2,
    ('NewJersey', 'Brooklyn'): 0.5,
    ('NewJersey', 'Queens'): 0.7,
    ('NewJersey', 'StatenIsland'): 0.4,
    ('NewJersey', 'C1'): 2.2,
    ('Bronx', 'C2'): 1.7,
    ('Bronx', 'C3'): 0.7,
    ('Bronx', 'C4'): 1.7,
    ('Bronx', 'C6'): 1.2,
    ('Brooklyn', 'C1'): 1.2,
    ('Brooklyn', 'C2'): 0.7,
    ('Brooklyn', 'C3'): 0.7,
    ('Brooklyn', 'C4'): 1.2,
    ('Brooklyn', 'C5'): 0.7,
    ('Queens', 'C2'): 1.7,
    ('Queens', 'C3'): 2.2,
    ('Queens', 'C5'): 0.7,
    ('Queens', 'C6'): 1.7,
    ('StatenIsland', 'C3'): 0.4,
    ('StatenIsland', 'C4'): 1.7,
    ('StatenIsland', 'C5'): 0.7,
    ('StatenIsland', 'C6'): 1.7
})

#### Define and Deploy Model

In [11]:
model = gp.Model("SupplyNetworkDesign")
flow = model.addVars(arcs, obj = cost, name = "flow")

#### Include Constraints

In [12]:
# Each generation center has capacity limits
centers = supply.keys()
center_flow = model.addConstrs((gp.quicksum(flow.select(center,"*")) <= supply[center] for center in centers), name = "center")

In [13]:
# landfills can node handle mor ethan their demand
landfills = demand.keys()
landfill_flow = model.addConstrs((gp.quicksum(flow.select("*", landfill)) == demand[landfill] for landfill in landfills), name = "landfill")

In [15]:
#the total ammound entering each of the four depots should be equal to the ammount leaving them
depots = through.keys()
depot_flow = model.addConstrs((gp.quicksum(flow.select(depot, "*")) == gp.quicksum(flow.select("*", depot)) for depot in depots), name = "depot")
# each depot can not handle more than its capacity
depot_capacity = model.addConstrs((gp.quicksum(flow.select("*", depot)) <= through[depot] for depot in depots), name = "depot_capacity")

#### Optimize the Model

In [16]:
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 165U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 20 rows, 29 columns and 77 nonzeros
Model fingerprint: 0xfecc02ac
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 4e+05]
Presolve removed 13 rows and 15 columns
Presolve time: 0.01s
Presolved: 7 rows, 14 columns, 23 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9900000e+05   1.999000e+04   0.000000e+00      0s
       4    5.4100000e+05   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.410000000e+05


#### Analyze the result
The demand for all landfills can be satisfied at a total cost of $541,000. Below is the optimal plan for waste flow:

In [22]:
product_flow = pd.DataFrame(columns=["From", "To", "Flow"])
for arc in arcs:
    if flow[arc].x > 1e-6:
        product_flow.loc[len(product_flow.index)+1,:] = {"From": arc[0], "To": arc[1], "Flow": flow[arc].x} 
product_flow.index=[''] * len(product_flow)
product_flow

,From,To,Flow
,NewYork,StatenIsland,80000.0
,NewYork,C1,100000.0
,NewYork,C6,40000.0
,NewJersey,Brooklyn,100000.0
,NewJersey,Queens,110000.0
,Brooklyn,C2,20000.0
,Brooklyn,C4,70000.0
,Brooklyn,C5,10000.0
,Queens,C5,110000.0
,StatenIsland,C3,80000.0


### The End!